In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import pandas as pd
import time
import json
import h5py

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
!nvidia-smi

Thu Nov 26 16:01:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    12W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 訓練期間的batch大小
batch_size = 16

# 測試期間的batch大小
test_batch_size = 16

# 訓練圖像的空間大小。所有圖像將使用resize調整為此大小。
image_size = 224

# label種類數
class_num = 11

# 訓練epochs的大小
num_epochs = 10

# 初始學習速率
lr = 0.01

# 你的資料夾路徑
dir_path = './drive/My Drive/HW2/'

In [5]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
model2 = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model2.roi_heads.box_predictor.cls_score.in_features
model2.roi_heads.box_predictor = FastRCNNPredictor(in_features, class_num)
model2 = model2.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [8]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model2.load_state_dict(checkpoint['state_dict'], strict=False)

    return model2
model_test = load_checkpoint(dir_path+'checkpoint.pth')

In [10]:
# 取得該資料夾中所有檔案名
tid = []
test_id = os.listdir(dir_path + '/test')
for i in range(len(test_id)):
    tid.append(i+1)

# 處理圖片路徑
testing_imgs = [os.path.join(dir_path + 'test', str(i)+'.png') for i in tid]
print(len(testing_imgs))
print(testing_imgs)

13068
['./drive/My Drive/HW2/test/1.png', './drive/My Drive/HW2/test/2.png', './drive/My Drive/HW2/test/3.png', './drive/My Drive/HW2/test/4.png', './drive/My Drive/HW2/test/5.png', './drive/My Drive/HW2/test/6.png', './drive/My Drive/HW2/test/7.png', './drive/My Drive/HW2/test/8.png', './drive/My Drive/HW2/test/9.png', './drive/My Drive/HW2/test/10.png', './drive/My Drive/HW2/test/11.png', './drive/My Drive/HW2/test/12.png', './drive/My Drive/HW2/test/13.png', './drive/My Drive/HW2/test/14.png', './drive/My Drive/HW2/test/15.png', './drive/My Drive/HW2/test/16.png', './drive/My Drive/HW2/test/17.png', './drive/My Drive/HW2/test/18.png', './drive/My Drive/HW2/test/19.png', './drive/My Drive/HW2/test/20.png', './drive/My Drive/HW2/test/21.png', './drive/My Drive/HW2/test/22.png', './drive/My Drive/HW2/test/23.png', './drive/My Drive/HW2/test/24.png', './drive/My Drive/HW2/test/25.png', './drive/My Drive/HW2/test/26.png', './drive/My Drive/HW2/test/27.png', './drive/My Drive/HW2/test/28.

In [11]:
class TestDataset(Dataset):
    def __init__(self, test_imgs, transform=None, target_transform=None):
        self.imgs = test_imgs
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        img = Image.open(self.imgs[index]).convert('RGB')
        b = np.array(img)
        original_height = b.shape[0]
        original_width = b.shape[1]

        if self.transform:
            img = self.transform(img)

        return img, original_height, original_width

    def __len__(self):
        return len(self.imgs)

In [12]:
test_data = TestDataset(test_imgs=testing_imgs,
                        transform=transforms.Compose([
                          transforms.Resize((300, 300)),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5),
                                               (0.5, 0.5, 0.5))
                          ]))

test_loader = DataLoader(test_data, batch_size=16)
print(len(test_loader))

817


In [13]:
def ResizetoOrigin(pred, origin):
    for i in range(len(pred)):
        for j in range(len(pred[i]['boxes'])):
            # [x1, y1, x2, y2] -> [y1, x1, y2, x2]
            y1 = pred[i]['boxes'][j][1]/300 * origin[i][0]
            x1 = pred[i]['boxes'][j][0]/300 * origin[i][1]
            y2 = pred[i]['boxes'][j][3]/300 * origin[i][0]
            x2 = pred[i]['boxes'][j][2]/300 * origin[i][1]
            pred[i]['boxes'][j][0] = y1
            pred[i]['boxes'][j][1] = y2
            pred[i]['boxes'][j][2] = x1
            pred[i]['boxes'][j][3] = x2

    new_pred = []
    for i in range(len(pred)):
        temp = []
        for j in pred[i]['boxes']:
            temp.append(j.cpu().tolist())
        new_pred.append({'bbox': temp,
                         'score': pred[i]['scores'].cpu().tolist(),
                         'label': pred[i]['labels'].cpu().tolist()})

    return new_pred

In [14]:
prediction = []
origin_sizes = []

In [15]:
def test(model, prediction, origin_sizes):
    since = time.time()
    model.eval()
    besttime = 10000

    with torch.no_grad():
        for i, (imgs, H, W) in enumerate(test_loader):
            imgs = imgs.to(device)

            # 取得預測機率
            outputs = model(imgs)

            for k in outputs:
                prediction.append(k)
            for l in range(len(H)):
                origin_sizes.append([H[l], W[l]])

            now_time = time.time() - since
            print("{}/{} test time is:{:.0f}m {:.0f}s".format(int(i+1),
                                                              len(test_loader),
                                                              now_time//60,
                                                              now_time % 60))
            since = time.time()

In [16]:
test(model_test, prediction, origin_sizes)

1/817 test time is:0m 4s
2/817 test time is:0m 4s
3/817 test time is:0m 4s
4/817 test time is:0m 4s
5/817 test time is:0m 4s
6/817 test time is:0m 4s
7/817 test time is:0m 4s
8/817 test time is:0m 4s
9/817 test time is:0m 4s
10/817 test time is:0m 4s
11/817 test time is:0m 4s
12/817 test time is:0m 4s
13/817 test time is:0m 4s
14/817 test time is:0m 4s
15/817 test time is:0m 4s
16/817 test time is:0m 4s
17/817 test time is:0m 4s
18/817 test time is:0m 4s
19/817 test time is:0m 4s
20/817 test time is:0m 4s
21/817 test time is:0m 4s
22/817 test time is:0m 4s
23/817 test time is:0m 4s
24/817 test time is:0m 4s
25/817 test time is:0m 4s
26/817 test time is:0m 4s
27/817 test time is:0m 4s
28/817 test time is:0m 4s
29/817 test time is:0m 4s
30/817 test time is:0m 4s
31/817 test time is:0m 4s
32/817 test time is:0m 4s
33/817 test time is:0m 4s
34/817 test time is:0m 4s
35/817 test time is:0m 4s
36/817 test time is:0m 4s
37/817 test time is:0m 4s
38/817 test time is:0m 4s
39/817 test time is:0

In [17]:
new_prediction = ResizetoOrigin(prediction, origin_sizes)

In [21]:
print(prediction[0])

{'boxes': tensor([[ 4.7965, 26.5329, 53.9359, 80.7685]], device='cuda:0'), 'labels': tensor([5], device='cuda:0'), 'scores': tensor([0.9923], device='cuda:0')}


In [19]:
with open(os.path.join(dir_path, "result.json"),"w") as f:
    json.dump(new_prediction, f)